In [1]:
import sqlite3
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
with open('my_cwb_apikey.txt') as f:
    apikey = f.readline()
format_ = 'json'
dataid = 'F-C0032-001'   # 一般天氣預報-今明36小時天氣預報

url = f'https://opendata.cwb.gov.tw/fileapi/v1/opendataapi/{dataid}?Authorization={apikey}&format={format_}'
resp = requests.get(url)
data = resp.json()

In [3]:
issue_time = data['cwbopendata']['dataset']['datasetInfo']['issueTime']
update = data['cwbopendata']['dataset']['datasetInfo']['update']
locations = data['cwbopendata']['dataset']['location']
print(issue_time)
print(update)

2019-08-27T23:00:00+08:00
2019-08-27T22:39:05+08:00


In [4]:
df = pd.DataFrame()

for loc in locations:
    loc_name = loc['locationName']
    elements = loc['weatherElement']
    for element in elements:
        element_name = element['elementName']
        for etime in element['time']:
            start_time = etime['startTime']
            end_time = etime['endTime']
            parameter = etime['parameter']['parameterName']
            df = df.append([[issue_time, loc_name, element_name, start_time, end_time, parameter]])
            
df = df.reset_index(drop=True)
df.columns = ['issueTime', 'location', 'element', 'startTime', 'endTime', 'value']
df.head(8)

,issueTime,location,element,startTime,endTime,value
0,2019-08-27T23:00:00+08:00,臺北市,Wx,2019-08-28T00:00:00+08:00,2019-08-28T06:00:00+08:00,晴時多雲
1,2019-08-27T23:00:00+08:00,臺北市,Wx,2019-08-28T06:00:00+08:00,2019-08-28T18:00:00+08:00,晴時多雲
2,2019-08-27T23:00:00+08:00,臺北市,Wx,2019-08-28T18:00:00+08:00,2019-08-29T06:00:00+08:00,晴時多雲
3,2019-08-27T23:00:00+08:00,臺北市,MaxT,2019-08-28T00:00:00+08:00,2019-08-28T06:00:00+08:00,29
4,2019-08-27T23:00:00+08:00,臺北市,MaxT,2019-08-28T06:00:00+08:00,2019-08-28T18:00:00+08:00,36
5,2019-08-27T23:00:00+08:00,臺北市,MaxT,2019-08-28T18:00:00+08:00,2019-08-29T06:00:00+08:00,32
6,2019-08-27T23:00:00+08:00,臺北市,MinT,2019-08-28T00:00:00+08:00,2019-08-28T06:00:00+08:00,27
7,2019-08-27T23:00:00+08:00,臺北市,MinT,2019-08-28T06:00:00+08:00,2019-08-28T18:00:00+08:00,27


In [5]:
# create db
conn = sqlite3.connect('county_36hr_forecast.sqlite')
c = conn.cursor()
c.execute('CREATE TABLE county '
          + '(id INTEGER PRIMARY KEY AUTOINCREMENT, '
          + 'issueTime TEXT, location TEXT, element TEXT, '
          + 'startTime TEXT, endTime TEXT, value TEXT)')
conn.commit()
conn.close()

In [6]:
# append data into db
conn = sqlite3.connect('county_36hr_forecast.sqlite')
df.to_sql('county', conn, if_exists='append', index=False)
conn.close()

In [8]:
# read data from db
conn = sqlite3.connect('county_36hr_forecast.sqlite')
c = conn.cursor()
c.execute('SELECT * FROM county')
rows = c.fetchall()
columns = [des[0] for des in c.description]
print(columns)

for row in rows[:100]:
    print(row)

conn.close()

['id', 'issueTime', 'location', 'element', 'startTime', 'endTime', 'value']
(1, '2019-08-27T23:00:00+08:00', '臺北市', 'Wx', '2019-08-28T00:00:00+08:00', '2019-08-28T06:00:00+08:00', '晴時多雲')
(2, '2019-08-27T23:00:00+08:00', '臺北市', 'Wx', '2019-08-28T06:00:00+08:00', '2019-08-28T18:00:00+08:00', '晴時多雲')
(3, '2019-08-27T23:00:00+08:00', '臺北市', 'Wx', '2019-08-28T18:00:00+08:00', '2019-08-29T06:00:00+08:00', '晴時多雲')
(4, '2019-08-27T23:00:00+08:00', '臺北市', 'MaxT', '2019-08-28T00:00:00+08:00', '2019-08-28T06:00:00+08:00', '29')
(5, '2019-08-27T23:00:00+08:00', '臺北市', 'MaxT', '2019-08-28T06:00:00+08:00', '2019-08-28T18:00:00+08:00', '36')
(6, '2019-08-27T23:00:00+08:00', '臺北市', 'MaxT', '2019-08-28T18:00:00+08:00', '2019-08-29T06:00:00+08:00', '32')
(7, '2019-08-27T23:00:00+08:00', '臺北市', 'MinT', '2019-08-28T00:00:00+08:00', '2019-08-28T06:00:00+08:00', '27')
(8, '2019-08-27T23:00:00+08:00', '臺北市', 'MinT', '2019-08-28T06:00:00+08:00', '2019-08-28T18:00:00+08:00', '27')
(9, '2019-08-27T23:00:00+08: